# Tarea 5 - Árbol de Desición

***Inteligencia Artificial***

***II Semestre 2024***

***Tecnológico de Costa Rica***

Estudiantes: 
- Esteban Guzmán
- Rolando Mora

In [29]:
import pandas as pd
import numpy as np

In [23]:
# Carga de datos
data = pd.read_csv("adult-census.csv")
data

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


## Actividad - Taller

### 1. Cree una clase nodo con atributos necesarios para un árbol de decisión: feature, umbral, gini, cantidad_muestras, valor, izquierda, derecha

In [2]:
class Nodo:
    # feature: Índice de la característica utilizada para la división en este nodo.
    # umbral: Valor del umbral utilizado para la división.
    # gini: Índice de Gini de este nodo.
    # cantidad_muestras: Número de muestras en este nodo.
    # valor: Valor de la clase para nodos hoja (puede ser el valor promedio o la clase más común).
    # izquierda: Nodo hijo izquierdo.
    # derecha: Nodo hijo derecho.
    def __init__(self, feature, umbral, gini, cantidad_muestras, valor, izquierda, derecha):
        self.feature = feature
        self.umbral = umbral
        self.gini = gini
        self.cantidad_muestras = cantidad_muestras
        self.valor = valor
        self.izquierda = izquierda
        self.derecha = derecha

### 2. Crea una clase que implementa un árbol de decisión, utilice las funciones presentadas en clase, además incluya los siguientes hyperparámetros:
- max_depth: Cantidad máxima de variables que se pueden explorar
- min_split_samples: Cantidad mínima de muestras que deberá tener un nodo para poder ser dividido
- criterio: función que se utilizará para calcular la impuridad.

In [43]:
class ArbolDecision:
    # :param max_depth: Profundidad máxima del árbol.
    # :param min_split_samples: Número mínimo de muestras necesarias para dividir un nodo.
    # :param criterio: Criterio utilizado para calcular la impureza ("gini" o "entropia").
    def __init__(self, max_depth=1, min_split_samples=2, criterio="gini"):
        self.max_depth = max_depth
        self.min_split_samples = min_split_samples
        self.criterio = criterio
        self.raiz = None

    # Función que calcula el coeficiente de Gini
    def gini_impurity(self, labels):
        # Calculamos la proporción de cada clase
        proportions = labels.value_counts() / len(labels)
        # Calculamos el coeficiente de Gini
        impurity = 1 - sum(proportions**2)
        return impurity
    
    # Función que calcula la entropía
    def entropy_impurity(self, labels):
        # Calculamos la proporción de cada clase
        proportions = labels.value_counts() / len(labels)
        # Calculamos la entropía
        impurity = sum(-proportions * np.log2(proportions))
        return impurity
    
    # Método para calcular la ganancia de información al realizar un corte en los datos
    def ganancia_informacion(self, y, y_mask, impurity_func):
        a = sum(y_mask)  # Cantidad de elementos en la partición izquierda
        b = y_mask.shape[0] - a  # Cantidad de elementos en la partición derecha

        # Si una de las particiones es vacía, la ganancia de información es cero
        if a == 0 or b == 0:
            return 0

        # Calcular la impureza del nodo antes de la división
        impurity_general = impurity_func(y)
        # Calcular la impureza de las particiones izquierda y derecha
        impuridad_izquierda = impurity_func(y[y_mask])
        impuridad_derecha = impurity_func(y[~y_mask])

        # Proporciones de las particiones izquierda y derecha
        propor_izq = len(y[y_mask]) / len(y)
        propor_der = len(y[~y_mask]) / len(y)

        # Impureza después de realizar la división
        impuridad_al_corte = (propor_izq * impuridad_izquierda) + (propor_der * impuridad_derecha)
        # Calcular la ganancia de información como la diferencia entre la impureza original y la impureza después del corte
        gain = impurity_general - impuridad_al_corte

        return gain
    
    # Método para encontrar la mejor variable y punto de corte que maximice la ganancia de información
    def buscar_mejor_information_gain_variable(self, x, y, criterio):
        # Obtener valores únicos ordenados
        x_unique = np.sort(np.unique(x))
        midpoints = (x_unique[:-1] + x_unique[1:]) / 2  # Calcular puntos medios para posibles divisiones

        max_gain = 0
        best_midpoint = None

        # Iterar sobre cada posible punto de corte
        for midpoint in midpoints:
            izquierda = y[x <= midpoint]
            derecha = y[x > midpoint]

            # Ignorar divisiones que no cumplen con el número mínimo de muestras
            if len(izquierda) < self.min_split_samples or len(derecha) < self.min_split_samples:
                continue

            # Calcular la ganancia de información para la división actual
            gain = self.ganancia_informacion(y, x > midpoint, impurity_func=criterio)

            # Si la ganancia es mayor que la máxima encontrada, actualizar los valores
            if gain > max_gain:
                max_gain = gain
                best_midpoint = midpoint

        return max_gain, best_midpoint
    
    # Método recursivo para construir el árbol de decisión
    def _construir_arbol(self, X, y, depth):
        print(X.shape)
        num_samples, num_features = X.shape
        criterio = self.gini_impurity if self.criterio == "gini" else self.entropy_impurity
        # Valor predominante en el nodo
        valor_nodo = y.mode()[0] if len(y.mode()) > 0 else y.mean()

        # Crear el nodo con la impureza, cantidad de muestras y valor predominante
        nodo = Nodo(gini=criterio(y), cantidad_muestras=num_samples, valor=valor_nodo)

        # Condiciones de parada: profundidad máxima alcanzada, pocas muestras o etiquetas homogéneas
        if depth >= self.max_depth or num_samples < self.min_split_samples or len(y.unique()) == 1:
            return nodo

        best_gain = 0
        best_feature = None
        best_threshold = None

        # Buscar la mejor variable y punto de corte para dividir
        for feature in range(num_features):
            x_feature = X.iloc[:, feature]
            gain, threshold = self.buscar_mejor_information_gain_variable(x_feature, y, criterio)
            if gain > best_gain:
                best_gain = gain
                best_feature = feature
                best_threshold = threshold

        # Si no se encuentra una ganancia significativa, devolver el nodo actual
        if best_gain == 0:
            return nodo

        # Crear los nodos hijos recursivamente
        indices_izq = X.iloc[:, best_feature] <= best_threshold
        izquierda = self._construir_arbol(X[indices_izq], y[indices_izq], depth + 1)
        derecha = self._construir_arbol(X[~indices_izq], y[~indices_izq], depth + 1)

        # Asignar las características de la división al nodo actual
        nodo.feature = best_feature
        nodo.umbral = best_threshold
        nodo.izquierda = izquierda
        nodo.derecha = derecha

        return nodo

    # Método para ajustar el árbol de decisión a los datos de entrenamiento
    def fit(self, X, y):
        self.raiz = self._construir_arbol(X, y, 0)

    # Método para predecir el valor para un solo ejemplo recorriendo el árbol
    def _predecir_individual(self, x, nodo):
        # Si el nodo es hoja, devolver el valor
        if nodo.izquierda is None and nodo.derecha is None:
            return nodo.valor

        # Elegir la rama izquierda o derecha según el umbral
        if x[nodo.feature] <= nodo.umbral:
            return self._predecir_individual(x, nodo.izquierda)
        else:
            return self._predecir_individual(x, nodo.derecha)

    # Método para predecir múltiples ejemplos
    def predict(self, X):
        return X.apply(lambda x: self._predecir_individual(x, self.raiz), axis=1)

### 3. Divida los datos en los conjuntos tradicionales de entrenamiento y prueba, de forma manual, sin utilizar las utilidades de sklearn (puede utilizar índices de Numpy o Pandas)

In [4]:
def dividir_datos(X, y, proporción=0.8):
    indices = np.random.permutation(len(X))
    limite = int(len(X) * proporción)

    X_entrenamiento, y_entrenamiento = X[indices[:limite]], y[indices[:limite]]
    X_prueba, y_prueba = X[indices[limite:]], y[indices[limite:]]

    return X_entrenamiento, X_prueba, y_entrenamiento, y_prueba

### 4. Entrene 10 combinaciones distintas de parámetros para su implementación de Arbol de Decisión (puede utilizar como ejemplo la funcionalidad de GridSearch de SKLearn).

In [44]:
X = data["age"]
y = data["class"]

X_train, X_test, y_train, y_test = dividir_datos(X, y)

param_grid = [
    {"max_depth": 3, "min_split_samples": 2, "criterio": "gini"},
    {"max_depth": 5, "min_split_samples": 4, "criterio": "gini"},
    {"max_depth": 7, "min_split_samples": 6, "criterio": "gini"},
    {"max_depth": 3, "min_split_samples": 2, "criterio": "entropia"},
    {"max_depth": 5, "min_split_samples": 4, "criterio": "entropia"},
    {"max_depth": 7, "min_split_samples": 6, "criterio": "entropia"},
    {"max_depth": 10, "min_split_samples": 2, "criterio": "gini"},
    {"max_depth": 10, "min_split_samples": 5, "criterio": "gini"},
    {"max_depth": 3, "min_split_samples": 10, "criterio": "entropia"},
    {"max_depth": 10, "min_split_samples": 10, "criterio": "entropia"},
]

for params in param_grid:
    modelo = ArbolDecision(
        max_depth=params["max_depth"],
        min_split_samples=params["min_split_samples"],
        criterio=params["criterio"]
    )
    modelo.fit(X_train, y_train)
    

(39073,)


ValueError: not enough values to unpack (expected 2, got 1)

## Pruebas

In [6]:
# Agrupación por clase y por adultez
data["umbral"] = data["age"] > 30
data.groupby(["umbral", "class"]).size().unstack("class")

class,<=50K,>50K
umbral,,
False,14800,993
True,22355,10694


In [7]:
# Clasificación de muestras
print(
    "Muestras mal clasificadas con un corte en 25:",
    data.loc[(data["age"]>=25) & (data["class"]==" <=50K"),:].shape[0], "\n",
    "Muestras mal clasificadas con un corte en 50:",
    data.loc[(data["age"]>=50) & (data["class"]==" <=50K"),:].shape[0]
)

Muestras mal clasificadas con un corte en 25: 28816 
 Muestras mal clasificadas con un corte en 50: 7180


In [8]:
# Pruebas gini

# Gini para el dataset completo
print(ArbolDecision().gini_impurity(data["class"]))

# Gini para el dataset dividido por el umbral
print(ArbolDecision().gini_impurity(data[data["age"] > 50]["class"]))

0.36405200460016074
0.43390451896643945


In [9]:
# Prueba entropía

# Se calcula la entropía de los adultos mayores a 50 años
ArbolDecision().entropy_impurity(data[data["age"] > 50]["class"])

0.9024238545883967

### Prueba de ganancia de información

In [31]:
arbol = ArbolDecision()
arbol.ganancia_informacion(data["class"], data["age"] <= 50, impurity_func=arbol.gini_impurity)

0.0031306033146626944

### Prueba de mejor ganancia de información

In [38]:
arbol = ArbolDecision()
arbol.buscar_mejor_information_gain_variable(data["age"], data["class"], criterio=arbol.gini_impurity)

(0.02985403689151772, 29.5)

La implementación manual del árbol de decisión ayudó a comprender cómo se construye y optimiza este modelo. Permite apreciar el proceso de selección de características y criterios de división. Fue un ejercicio valioso para entender sus fundamentos y desafíos.